In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./file/2.make.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Year'] = input_['Year'].astype(int)
input_['Year Code'] = input_['Year Code'].astype(int)
input_['Make Code'] = input_['Make Code'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =


work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            b = 0
            while True:
                b += 1
                
                try:
                    resp = requests.get(f'''https://www.auveco.com/store_ss2/extensions/UnlockCommerce/YearMakeModel/1.0.0/Modules/YearMakeModel/SuiteScript2/YearMakeModel.Service.ss?action=getmodels&make={input_.loc[a, 'Make Code']}&year={input_.loc[a, 'Year Code']}''',
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(10, 10))

                    if resp.status_code == 200:
                        json_ = resp.json()

                        list_model = [dict_['model'] if dict_['model'] else '' for dict_ in json_]
                        while '' in list_model:
                            list_model.remove('')
                        if list_model:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame({'Make': input_.loc[a, 'Make'],
                                    'Make Code': input_.loc[a, 'Make Code'],
                                    'Model': list_model,
                                    'Year': input_.loc[a, 'Year'],
                                    'Year Code': input_.loc[a, 'Year Code']})
            
            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'Make']} > {input_.loc[a, 'Year']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Make', 'Model', 'Year'],
                                                ascending=[True, True, False],
                                                ignore_index=True)
    output_correct.to_excel(f'''./file/3.model-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Make', 'Year'],
                                            ascending=[True, False],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/model_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：5] - 2009 > FORD
[剩余数量：1000] - [当前时间：15:53:25]

[状态：ok，尝试次数：50] - 2017 > FIAT
[剩余数量：999] - [当前时间：15:53:25]

[状态：ok，尝试次数：1] - 2002 > FORD
[剩余数量：998] - [当前时间：15:53:26]

[状态：ok，尝试次数：17] - 2014 > FORD
[剩余数量：997] - [当前时间：15:53:26]

[状态：ok，尝试次数：25] - 1985 > DODGE
[剩余数量：996] - [当前时间：15:53:26]

[状态：ok，尝试次数：83] - 1988 > DODGE
[剩余数量：995] - [当前时间：15:53:26]

[状态：ok，尝试次数：2] - 2000 > FORD
[剩余数量：994] - [当前时间：15:53:26]

[状态：ok，尝试次数：2] - 1999 > FORD
[剩余数量：993] - [当前时间：15:53:26]

[状态：ok，尝试次数：42] - 2023 > FORD
[剩余数量：992] - [当前时间：15:53:27]

[状态：ok，尝试次数：17] - 2005 > FORD
[剩余数量：991] - [当前时间：15:53:27]

[状态：ok，尝试次数：29] - 2015 > FORD
[剩余数量：990] - [当前时间：15:53:28]

[状态：ok，尝试次数：107] - 1991 > DODGE
[剩余数量：989] - [当前时间：15:53:28]

[状态：ok，尝试次数：12] - 1996 > FORD
[剩余数量：988] - [当前时间：15:53:28]

[状态：ok，尝试次数：4] - 1989 > FORD
[剩余数量：987] - [当前时间：15:53:28]

[状态：ok，尝试次数：4] - 1986 > FORD
[剩余数量：986] - [当前时间：15:53:28]

[状态：ok，尝试次数：96] - 2000 > DODGE
[剩余数量：985] - [当前时间：15:53:28]

[状态：ok，尝试次数：61] - 2009 > DODGE
[剩余数量：984